# Nyc taxi data regression with pipeline component

**Requirements** - In order to benefit from this tutorial, you will need:
- A basic understanding of Machine Learning
- An Azure account with an active subscription - [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace with computer cluster - [Configure workspace](../../configuration.ipynb)
- A python environment
- Installed Azure Machine Learning Python SDK v2 - [install instructions](../../../README.md) - check the getting started section

**Learning Objectives** - By the end of this tutorial, you should be able to:
- Connect to your AML workspace from the Python SDK
- Create `PipelineComponent` using loaded component.
- Create `PipelineJob` with `PipelineComponent`.

**Motivations** - This notebook covers the scenario that user define pipeline component using dsl decorator.

# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1 Import the required libraries

In [ ]:
# Import required libraries
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

from azure.ai.ml import MLClient, Input
from azure.ai.ml.dsl import pipeline
from azure.ai.ml import load_component

import os 
os.environ["AZURE_ML_CLI_PRIVATE_FEATURES_ENABLED"] = "true"

## 1.2 Configure credential

We are using `DefaultAzureCredential` to get access to workspace. 
`DefaultAzureCredential` should be capable of handling most Azure SDK authentication scenarios. 

Reference for more available credentials if it does not work for you: [configure credential example](../../configuration.ipynb), [azure-identity reference doc](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python).

In [ ]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

## 1.3 Get a handle to the workspace

We use config file to connect to a workspace. The Azure ML workspace should be configured with computer cluster. [Check this notebook for configure a workspace](../../configuration.ipynb)

In [ ]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

# Retrieve an already attached Azure Machine Learning Compute.
cluster_name = "cpu-cluster"
print(ml_client.compute.get(cluster_name))

# 2. Define and create components into workspace
## 2.1 Define pipeline component with data prep and transformation
### 2.1.1 Load components from YAML

In [ ]:
components_dir = "../../cli/nyc_taxi_data_regression_with_pipeline_component/data_pipeline"
prep = load_component(path=f"{components_dir}/prep_src/prep.yml")
transform = load_component(path=f"{components_dir}/transform_src/transform.yml")

### 2.1.2 Define pipeline component with components

In [ ]:
# add meta info when invoke decorator.
@pipeline(display_name='data_pipeline',
    tags={'authoring':'sdk'},
    # version='123',
    description='pipeline component with data prep and transformation defined via yaml-meta.',
    is_deterministic=True
    )
def data_pipeline(
    raw_data: Input,
):
    """pipeline component with data prep and transformation defined via yaml."""
    # Call component obj as function: apply given inputs & parameters to create a node in pipeline
    prep_node = prep(
        raw_data=raw_data,
    )
    transform_node = transform(
        clean_data=prep_node.outputs.prep_data
    )
    # Return: pipeline outputs
    return {
        "train_data": transform_node.outputs.transformed_data,
    }

### 2.1.3 (Optional) Create pipeline component to workspace

In [ ]:
registered_data_pipeline_component = ml_client.components.create_or_update(data_pipeline)

## 2.2 Define pipeline component pipeline component with train, predict and score
### 2.2.1 Load components from YAML

In [ ]:
components_dir = "../../cli/nyc_taxi_data_regression_with_pipeline_component/train_pipeline"
train = load_component(path=f"{components_dir}/train_src/train.yml")
predict = load_component(path=f"{components_dir}/predict_src/predict.yml")
score = load_component(path=f"{components_dir}/score_src/score.yml")

### 2.1.2 Define pipeline component with components

In [ ]:
@pipeline
def train_pipeline(
    train_data: Input,
    compute_train_node: str,
):
    """pipeline component with train, predict and score defined via yaml."""
    # Call component obj as function: apply given inputs & parameters to create a node in pipeline
    train_node = train(
        train_data=train_data,
    )
    train_node.compute=compute_train_node
    predict_node = predict(
        model_input=train_node.outputs.model_output,
        test_data=train_node.outputs.test_data
    )
    score_node=score(
        predictions=predict_node.outputs.predictions,
        model=train_node.outputs.model_output
    )
    # Return: pipeline outputs
    return {
        "trained_model": train_node.outputs.model_output,
        "predictions": predict_node.outputs.predictions,
        "score_report": score_node.outputs.score_report,

    }

### 2.1.3 (Optional) Create pipeline component to workspace

In [ ]:
registered_train_pipeline_component = ml_client.components.create_or_update(train_pipeline)

# 3. Build nyc_taxi_data_regression pipeline job
## 3.1 Build pipeline

In [ ]:
# Construct pipeline
@pipeline
def nyc_taxi_data_regression(
    pipeline_raw_data,
    compute_train_node,
):
    # Use data pipeline component
    data_pipeline_node = data_pipeline(
        raw_data=pipeline_raw_data, 
    )
    # Use train pipeline component
    train_pipeline_node = train_pipeline(
        train_data=data_pipeline_node.outputs.train_data, 
        compute_train_node=compute_train_node
    )
    # Return: pipeline outputs
    return {
        "pipeline_job_trained_model": train_pipeline_node.outputs.trained_model,
        "pipeline_job_predictions": train_pipeline_node.outputs.predictions,
        "pipeline_job_score_report": train_pipeline_node.outputs.score_report
    }


pipeline_job = nyc_taxi_data_regression(
    pipeline_raw_data=Input(type="uri_folder", path="../../cli/nyc_taxi_data_regression_with_pipeline_component//data/"),
    compute_train_node="gpu-cluster"
)

# set pipeline level compute
pipeline_job.settings.default_compute="cpu-cluster"


In [ ]:
# Inspect built pipeline
print(pipeline_job)

## 3.2 Submit pipeline job

In [ ]:
# Submit pipeline job to workspace
pipeline_job = ml_client.jobs.create_or_update(
    pipeline_job, experiment_name="nyc_taxi_data_regression_with_pipeline_component"
)
pipeline_job

In [ ]:
# Wait until the job completes
ml_client.jobs.stream(pipeline_job.name)

# Next Steps
You can see further examples of running a pipeline job [here](../README.md)